# Geospatial Analysis for Mortgage Risk Assessment: A Data-Driven Approach to Flood Risk Evaluation

##Executive Summary

In [1]:
import geopandas as gpd

# Specify the filepath to the shapefile directory
shapefile_dir = r'C:\Users\HP\Downloads\IndicativeFloodRiskAreas-SHP\data'

# Attempt to read the shapefile
try:
    flood_zones = gpd.read_file(shapefile_dir)
    # If successful, print the first few rows of the GeoDataFrame
    print(flood_zones.head())
except Exception as e:
    # If reading fails, print the error message
    print("Error:", e)


    easting  northing    gridsq  numresp  numpeople  numkeys  numnonresp  \
0  165500.0   27500.0  X165Y027       96     224.64        0          18   
1  163500.0   39500.0  X163Y039        4       9.36        2           0   
2  151500.0   40500.0  X151Y040       70     163.80        1          58   
3  169500.0   41500.0  X169Y041       39      91.26        0          28   
4  182500.0   44500.0  X182Y044      133     311.22        5         263   

   st_area_sh  st_perimet                                           geometry  
0   1000000.0      4000.0  POLYGON ((166000.000 27000.000, 165000.000 270...  
1   1000000.0      4000.0  POLYGON ((164000.000 39000.000, 163000.000 390...  
2   1000000.0      4000.0  POLYGON ((152000.000 40000.000, 151000.000 400...  
3   1000000.0      4000.0  POLYGON ((170000.000 41000.000, 169000.000 410...  
4   1000000.0      4000.0  POLYGON ((183000.000 44000.000, 182000.000 440...  


In [3]:
# Print the entire GeoJSON data to inspect its structure and contents
print("GeoJSON data:", flood_zones.to_json())

# Create a map centered around the mean easting and northing coordinates
m = folium.Map(location=[flood_zones["northing"].mean(), flood_zones["easting"].mean()], zoom_start=10)

# Add GeoJson layer for flood zones
folium.GeoJson(flood_zones).add_to(m)

# Display the map
m


GeoJSON data: {"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"easting": 165500.0, "northing": 27500.0, "gridsq": "X165Y027", "numresp": 96, "numpeople": 224.64, "numkeys": 0, "numnonresp": 18, "st_area_sh": 1000000.0, "st_perimet": 4000.0}, "geometry": {"type": "Polygon", "coordinates": [[[166000.0, 27000.0], [165000.0, 27000.0], [165000.0, 28000.0], [166000.0, 28000.0], [166000.0, 27000.0]]]}}, {"id": "1", "type": "Feature", "properties": {"easting": 163500.0, "northing": 39500.0, "gridsq": "X163Y039", "numresp": 4, "numpeople": 9.36, "numkeys": 2, "numnonresp": 0, "st_area_sh": 1000000.0, "st_perimet": 4000.0}, "geometry": {"type": "Polygon", "coordinates": [[[164000.0, 39000.0], [163000.0, 39000.0], [163000.0, 40000.0], [164000.0, 40000.0], [164000.0, 39000.0]]]}}, {"id": "2", "type": "Feature", "properties": {"easting": 151500.0, "northing": 40500.0, "gridsq": "X151Y040", "numresp": 70, "numpeople": 163.8, "numkeys": 1, "numnonresp": 58, "s

In [4]:
import pandas as pd
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim

# Sample mortgage data with fictive values and real addresses
mortgages = pd.DataFrame({
    "address": [
        "1 Town Square, Barking, IG11 7LU",  # Barking and Dagenham
        "St Ives Road, Maidenhead, SL6 1RF",  # Windsor and Maidenhead
        "Cardiff Bay, Cardiff, CF10"  # Cardiff
    ],
    "loan_amount": [250000, 400000, 175000],
    "property_value": [300000, 500000, 200000]
})

# Geocode addresses
geocoder = Nominatim(user_agent="your_app_name")
for index, row in mortgages.iterrows():
    location = geocoder.geocode(row["address"])
    if location:
        mortgages.loc[index, "latitude"] = location.latitude
        mortgages.loc[index, "longitude"] = location.longitude
    else:
        mortgages.loc[index, "latitude"] = None
        mortgages.loc[index, "longitude"] = None

# Create a map centered around the mean latitude and longitude
m = folium.Map(location=[mortgages["latitude"].mean(), mortgages["longitude"].mean()], zoom_start=10)

# Add markers for each property
for _, row in mortgages.iterrows():
    if not pd.isnull(row["latitude"]) and not pd.isnull(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"Loan Amount: {row['loan_amount']}, Property Value: {row['property_value']}"
        ).add_to(m)

# Display the map
m


In [5]:
import pandas as pd
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim
import random

# Sample mortgage data with fictive values and real addresses
mortgages = pd.DataFrame({
    "address": [
        "1 Town Square, Barking, IG11 7LU",  # Barking and Dagenham
        "St Ives Road, Maidenhead, SL6 1RF",  # Windsor and Maidenhead
        "Cardiff Bay, Cardiff, CF10"  # Cardiff
    ],
    "loan_amount": [250000, 400000, 175000],
    "property_value": [300000, 500000, 200000]
})

# Geocode addresses
geocoder = Nominatim(user_agent="your_app_name")
for index, row in mortgages.iterrows():
    location = geocoder.geocode(row["address"])
    if location:
        mortgages.loc[index, "latitude"] = location.latitude
        mortgages.loc[index, "longitude"] = location.longitude
    else:
        mortgages.loc[index, "latitude"] = None
        mortgages.loc[index, "longitude"] = None

# Example function to simulate flood risk
def simulate_flood_risk(property_type, proximity_to_water):
    """
    Simulates flood risk for a property.

    Args:
        property_type (str): Type of property (e.g., residential, commercial).
        proximity_to_water (float): Distance to water bodies (e.g., rivers, lakes).

    Returns:
        float: Estimated flood risk score (0 to 1).
    """
    # Assume higher risk for properties close to water
    base_risk = 0.3 if property_type == "residential" else 0.5
    risk_increase = proximity_to_water * 0.02

    # Add some randomness
    random_factor = random.uniform(-0.05, 0.05)

    # Calculate total risk
    total_risk = base_risk + risk_increase + random_factor
    return min(max(total_risk, 0), 1)  # Ensure risk score is between 0 and 1

# Add simulated flood risk column to mortgages DataFrame
mortgages["flood_risk"] = [simulate_flood_risk("residential", random.random()) for _ in range(len(mortgages))]

# Create a map centered around the mean latitude and longitude
m = folium.Map(location=[mortgages["latitude"].mean(), mortgages["longitude"].mean()], zoom_start=10)

# Add markers for each property with popup showing loan amount, property value, and flood risk
for _, row in mortgages.iterrows():
    if not pd.isnull(row["latitude"]) and not pd.isnull(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"Loan Amount: {row['loan_amount']}, Property Value: {row['property_value']}, Flood Risk: {row['flood_risk']:.2f}"
        ).add_to(m)

# Display the map
m


In [6]:
import pandas as pd
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim
import random

# Sample mortgage data with fictive values and real addresses
mortgages = pd.DataFrame({
    "address": [
        "1 Town Square, Barking, IG11 7LU",  # Barking and Dagenham
        "St Ives Road, Maidenhead, SL6 1RF",  # Windsor and Maidenhead
        "Cardiff Bay, Cardiff, CF10"  # Cardiff
    ],
    "loan_amount": [250000, 400000, 175000],
    "property_value": [300000, 500000, 200000]
})

# Geocode addresses
geocoder = Nominatim(user_agent="your_app_name")
for index, row in mortgages.iterrows():
    location = geocoder.geocode(row["address"])
    if location:
        mortgages.loc[index, "latitude"] = location.latitude
        mortgages.loc[index, "longitude"] = location.longitude
    else:
        mortgages.loc[index, "latitude"] = None
        mortgages.loc[index, "longitude"] = None

# Example function to simulate flood risk
def simulate_flood_risk(property_type, proximity_to_water):
    """
    Simulates flood risk for a property.

    Args:
        property_type (str): Type of property (e.g., residential, commercial).
        proximity_to_water (float): Distance to water bodies (e.g., rivers, lakes).

    Returns:
        float: Estimated flood risk score (0 to 1).
    """
    # Assume higher risk for properties close to water
    base_risk = 0.3 if property_type == "residential" else 0.5
    risk_increase = proximity_to_water * 0.02

    # Add some randomness
    random_factor = random.uniform(-0.05, 0.05)

    # Calculate total risk
    total_risk = base_risk + risk_increase + random_factor
    return min(max(total_risk, 0), 1)  # Ensure risk score is between 0 and 1

# Add simulated flood risk column to mortgages DataFrame
mortgages["flood_risk"] = [simulate_flood_risk("residential", random.random()) for _ in range(len(mortgages))]

def is_loan_at_risk(flood_risk):
    """
    Determines if a loan is at risk based on flood risk score.

    Args:
        flood_risk (float): Flood risk score (0 to 1).

    Returns:
        str: Risk assessment ('Low risk', 'Moderate risk', or 'High risk').
    """
    if flood_risk < 0.3:
        return 'Low risk'
    elif flood_risk < 0.7:
        return 'Moderate risk'
    else:
        return 'High risk'

# Add loan risk column to mortgages DataFrame
mortgages["loan_risk"] = mortgages["flood_risk"].apply(is_loan_at_risk)

# Print the risk assessment for each property
for _, row in mortgages.iterrows():
    print(f"Address: {row['address']}, Loan Risk: {row['loan_risk']}")

# Create a map centered around the mean latitude and longitude
m = folium.Map(location=[mortgages["latitude"].mean(), mortgages["longitude"].mean()], zoom_start=10)

# Add markers for each property with popup showing loan amount, property value, flood risk, and loan risk assessment
for _, row in mortgages.iterrows():
    if not pd.isnull(row["latitude"]) and not pd.isnull(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"Loan Amount: {row['loan_amount']}, Property Value: {row['property_value']}, Flood Risk: {row['flood_risk']:.2f}, Loan Risk: {row['loan_risk']}"
        ).add_to(m)

# Display the map
m


Address: 1 Town Square, Barking, IG11 7LU, Loan Risk: Low risk
Address: St Ives Road, Maidenhead, SL6 1RF, Loan Risk: Low risk
Address: Cardiff Bay, Cardiff, CF10, Loan Risk: Low risk


In [7]:
# Example function to calculate loss given default
def calculate_loss_given_default(loan_amount, flood_risk_score):
    """
    Calculates the potential loss given default on the loan.

    Args:
        loan_amount (float): The amount of the loan.
        flood_risk_score (float): The flood risk score associated with the property.

    Returns:
        float: Estimated loss given default (as a percentage of the loan amount).
    """
    # Define the base loss rate and adjust based on flood risk
    base_loss_rate = 0.05  # 5% base loss rate for loans
    risk_adjustment = flood_risk_score * 0.1  # Adjust by 10% for each 0.1 increase in flood risk score

    # Calculate the total loss given default
    loss_given_default = base_loss_rate + risk_adjustment
    return min(max(loss_given_default, 0), 1)  # Ensure loss rate is between 0 and 1

# Example usage
loan_amount = 250000
flood_risk_score = 0.8  # Example flood risk score for a property
loss_given_default = calculate_loss_given_default(loan_amount, flood_risk_score)
print(f"Loss given default: {loss_given_default:.2%}")

Loss given default: 13.00%


In [8]:
# Example function to calculate loss given default
def calculate_loss_given_default(loan_amount, flood_risk_score):
    """
    Calculates the potential loss given default on the loan.

    Args:
        loan_amount (float): The amount of the loan.
        flood_risk_score (float): The flood risk score associated with the property.

    Returns:
        float: Estimated loss given default (as a percentage of the loan amount).
    """
    # Define the base loss rate and adjust based on flood risk
    base_loss_rate = 0.05  # 5% base loss rate for loans
    risk_adjustment = flood_risk_score * 0.1  # Adjust by 10% for each 0.1 increase in flood risk score

    # Calculate the total loss given default
    loss_given_default = base_loss_rate + risk_adjustment
    return min(max(loss_given_default, 0), 1)  # Ensure loss rate is between 0 and 1

# Example usage
loan_amount = 250000
flood_risk_score = 0.8  # Example flood risk score for a property
loss_given_default = calculate_loss_given_default(loan_amount, flood_risk_score)
print(f"Loss given default: {loss_given_default:.2%}")

Loss given default: 13.00%


In [12]:
import pandas as pd
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim
import random

# Sample mortgage data with fictive values and real addresses
mortgages = pd.DataFrame({
    "address": [
        "1 Town Square, Barking, IG11 7LU",  # Barking and Dagenham
        "St Ives Road, Maidenhead, SL6 1RF",  # Windsor and Maidenhead
        "Cardiff Bay, Cardiff, CF10"  # Cardiff
    ],
    "loan_amount": [250000, 400000, 175000],
    "property_value": [300000, 500000, 200000]
})

# Input manual values for address, loan amount, and property value
address = input("Enter the address: ")
loan_amount = float(input("Enter the loan amount: "))
property_value = float(input("Enter the property value: "))

# Append input values to the mortgages DataFrame using pd.concat
new_mortgage = pd.DataFrame({"address": [address], "loan_amount": [loan_amount], "property_value": [property_value]})
mortgages = pd.concat([mortgages, new_mortgage], ignore_index=True)

# Geocode addresses
geocoder = Nominatim(user_agent="your_app_name")
for index, row in mortgages.iterrows():
    location = geocoder.geocode(row["address"])
    if location:
        mortgages.loc[index, "latitude"] = location.latitude
        mortgages.loc[index, "longitude"] = location.longitude
    else:
        mortgages.loc[index, "latitude"] = None
        mortgages.loc[index, "longitude"] = None

# Example function to simulate flood risk
def simulate_flood_risk(property_type, proximity_to_water):
    """
    Simulates flood risk for a property.

    Args:
        property_type (str): Type of property (e.g., residential, commercial).
        proximity_to_water (float): Distance to water bodies (e.g., rivers, lakes).

    Returns:
        float: Estimated flood risk score (0 to 1).
    """
    # Assume higher risk for properties close to water
    base_risk = 0.3 if property_type == "residential" else 0.5
    risk_increase = proximity_to_water * 0.02

    # Add some randomness
    random_factor = random.uniform(-0.05, 0.05)

    # Calculate total risk
    total_risk = base_risk + risk_increase + random_factor
    return min(max(total_risk, 0), 1)  # Ensure risk score is between 0 and 1

# Add simulated flood risk column to mortgages DataFrame
mortgages["flood_risk"] = [simulate_flood_risk("residential", random.random()) for _ in range(len(mortgages))]

def is_loan_at_risk(flood_risk):
    """
    Determines if a loan is at risk based on flood risk score.

    Args:
        flood_risk (float): Flood risk score (0 to 1).

    Returns:
        str: Risk assessment ('Low risk', 'Moderate risk', or 'High risk').
    """
    if flood_risk < 0.3:
        return 'Low risk'
    elif flood_risk < 0.7:
        return 'Moderate risk'
    else:
        return 'High risk'

# Add loan risk column to mortgages DataFrame
mortgages["loan_risk"] = mortgages["flood_risk"].apply(is_loan_at_risk)

# Calculate Loss Given Default (LGD)
mortgages["LGD"] = 1 - (mortgages["property_value"] - mortgages["loan_amount"]) / mortgages["property_value"]

# Print the risk assessment and LGD for each property
for _, row in mortgages.iterrows():
    print(f"Address: {row['address']}, Loan Risk: {row['loan_risk']}, LGD: {row['LGD']:.2f}")

# Create a map centered around the mean latitude and longitude
m = folium.Map(location=[mortgages["latitude"].mean(), mortgages["longitude"].mean()], zoom_start=10)

# Add markers for each property with popup showing loan amount, property value, flood risk, loan risk assessment, and LGD
for _, row in mortgages.iterrows():
    if not pd.isnull(row["latitude"]) and not pd.isnull(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"Loan Amount: {row['loan_amount']}, Property Value: {row['property_value']}, Flood Risk: {row['flood_risk']:.2f}, Loan Risk: {row['loan_risk']}, LGD: {row['LGD']:.2f}"
        ).add_to(m)

# Display the map
m


Enter the address: 89, Osmaston Road, Birmingham, B17 0TH
Enter the loan amount: 120000
Enter the property value: 89000
Address: 1 Town Square, Barking, IG11 7LU, Loan Risk: Moderate risk, LGD: 0.83
Address: St Ives Road, Maidenhead, SL6 1RF, Loan Risk: Moderate risk, LGD: 0.80
Address: Cardiff Bay, Cardiff, CF10, Loan Risk: Low risk, LGD: 0.88
Address: 89, Osmaston Road, Birmingham, B17 0TH, Loan Risk: Low risk, LGD: 1.35


###Descriptive Statistics